# Lecture9 Machine Translation and Advanced Recurrent LSTMs and GRUs

简单回顾了传统统计机器翻译中的难题，过渡到利用GRU和LSTM来救场，最后介绍了一些较新的改进工作。

## 传统统计机器翻译系统

对于情感分析这类还算简单的任务，你可以整理一个情感极性词典、编写一堆规则做出一个勉强能用的系统。但到了机器翻译这个高级应用，就无法完全依靠规则了。现代机器翻译手段都是基于统计的，在平行语料上学习语言知识。

定义原文为$f$，译文为$e$，机器翻译定义为找到使如下条件概率最大的$e$:$$e ^ { \hat { \imath } } = \operatorname { argmax } _ { e } p ( e | f ) = \operatorname { argmax } _ { e } p ( f | e ) p ( e )$$

这里利用了贝叶斯公式。翻译模型$p(f|e)$在平行语料上训练得到，语言模型$p(e)$在未对齐的原文语料上训练（是非常廉价的）。

公式描述的翻译过程如下：

1. 对齐

    找到原文中的哪个句子或短语翻译到译文中的哪个句子或短语，具体方法可以参考[Michael Collins NLP公开课任务3 机器翻译](http://www.hankcs.com/nlp/michael-collins-machine-translation.html)
    
    原文与译文单词之间可能存在一对一、一对多、多对一、多对多的对应关系，原文中有些单词还可能无明确译文，或译文中需要对一些单词进行补充
    
    ![1549785459845.jpg](https://i.loli.net/2019/02/10/5c5fd98fe7ab4.jpg)
    
    所有的对齐方式数量是组合数级的。有时候还要通过句法分析，来进行不同颗粒度的对齐：
    
    ![屏幕快照 2019-02-10 15.59.10.png](https://i.loli.net/2019/02/10/5c5fd9de750a8.png)
    
2. 短语组合

    原文中每个单词都有多个备选单词，导致了许多短语的组合方式：
    
    ![屏幕快照 2019-02-10 16.00.54.png](https://i.loli.net/2019/02/10/5c5fda56ae9c0.png)
    
3. 解码：在海量假设中搜索最佳选择

    这是一个特别复杂的搜索问题，涉及到许多语言模型。
    
    ![屏幕快照 2019-02-10 16.01.14.png](https://i.loli.net/2019/02/10/5c5fda569c1eb.png)
    
传统机器翻译系统需要人工做大量的特征工程，其中每个环节都是独立不同的机器学习问题。这些独立的模型各自为政，并不以一个统一的优化目标为最终目标。

而深度学习则提供了一个统一的模型，一个统一的最终目标函数。在优化目标函数的过程中，得到一个end to end的完整的joint模型。传统机器翻译系统与深度学习是截然相反的，对齐模型、词序模型、语言模型……一堆独立的模型无法联合训练。



## DL的MT解决方案

直接用RNN来接受原文，预测译文“下一个单词”：

![屏幕快照 2019-02-10 16.10.05.png](https://i.loli.net/2019/02/10/5c5fdc782a8b0.png)

红圈所示特征表示必须能捕捉整个原文短语的语义，但是RNN无法记住太久之前的事情，大概五六个单词就到极限了。所以这不是个实用的模型。

在这个最简单的模型中，

Encoder是：$$h _ { t } = \phi \left( h _ { t - 1 } , x _ { t } \right) = f \left( W ^ { ( h h ) } h _ { t - 1 } + W ^ { ( h x ) } x _ { t } \right)$$

Decoder是：$$h _ { t } = \phi \left( h _ { t - 1 } \right) = f \left( W ^ { ( h h ) } h _ { t - 1 } \right)$$
$$y _ { t } = \operatorname { softmax } \left( W ^ { ( S ) } h _ { t } \right)$$

$\phi \left( v _ { 1 } , v _ { 2 } \right)$一般代表接受的两个向量的权值不同。

softmax分类器中必须有个代表句子终止的“单词”，不然模型会无休止地输出下去。

但神经网络机器翻译模型没有这么简单，必须加一些拓展。

1. 编码器和解码器训练不同的权值矩阵

    红蓝代表不同的权值。
    
    ![屏幕快照 2019-02-10 16.21.42.png](https://i.loli.net/2019/02/10/5c5fe0a3b4782.png)
    
2. decoder中的隐藏层的输入来自3个方面：

    - 前一个时刻的隐藏层
    - encoder的最后一个隐藏层($c = h _ { T }$)
    - 前一个预测结果$\hat { y } _ { t - 1 } ^ { \wedge }$
    
    这样导致decoder函数变为：$h _ { t } = \phi \left( h _ { t - 1 } , c , y _ { t - 1 } \right)$
    
    ![屏幕快照 2019-02-10 17.05.20.png](https://i.loli.net/2019/02/10/5c5fe961e1be4.png)

    这可以辅助训练softmax的权值矩阵，防止模型重复生成同一个单词。

    上图还有一个复杂版本，表达的是同一个意思：
    
    ![屏幕快照 2019-02-10 17.06.14.png](https://i.loli.net/2019/02/10/5c5fe9bbbe541.png)
    
    其中，带黑点的表示离散的向量表示，否则表示连续的向量空间。
    
3. 使用深度RNN

    ![屏幕快照 2019-02-10 17.09.04.png](https://i.loli.net/2019/02/10/5c5fea7087c2b.png)
    
4. 使用 bi-directional encoder

    不再用$A B C→X Y$作为训练实例，而是逆转原文词序：$C B A→X Y$。因为A更可能翻译为X，而梯度消失导致A无法影响输出，倒过来A离输出近一些。逆转词序不会带来“语法语义上的改变”，因为模型学习的就是如何从逆序的原文翻译顺序的译文。但相应的，C就离Y更远了

## Gated Recurrent Units (GRU)

主要改进：更好的单元

引入GRU，这种单元可以让模型学习何时遗忘从而将记忆保持很久、允许误差根据输入的不同而不同。

标准的RNN直接计算隐藏层：$$h _ { t } = \sigma \left( W ^ { ( h h ) } h _ { t - 1 } + W ^ { ( h x ) } x _ { t } \right)$$

GRU先根据当前输入的词向量和隐藏层计算一个update gate（另一个层）：$$z _ { t } = \sigma \left( W ^ { ( z ) } x _ { t } + U ^ { ( z ) } h _ { t - 1 } \right)$$

利用相同的方法不同的权值计算reset gate:$$r _ { t } = \sigma \left( W ^ { ( r ) } x _ { t } + U ^ { ( r ) } h _ { t - 1 } \right)$$

然后利用reset gate计算新的记忆:$$h _ { t } ^ { \sim } = \tanh \left( r _ { t } \circ U h _ { t - 1 } + W x _ { t } \right)$$

这里的意思是，之前的记忆由reset gate控制，如果reset gate元素都是0，则遗忘之前的事情。比如电影评论的情感分析，“有个文艺的少男爱死了一个平凡的少女，这个平凡的少女也爱死了那个文艺的少男，可两个人就是无法相会巴拉巴拉，真是个无聊的电影”。无论前面说了多少话，起决定性作用的可能就是“无聊”这个词。那么一些GRU可能会说，我遇到了一个情感强烈的词语，我不想让它被之前的记忆冲淡（求和），所以我把reset gate设为0，之前的记忆不起作用，把这个情感词汇写成新的记忆。

而update gate的作用是调节最后的更新，到底时刻t的记忆多少来自之前，多少来自当前：$$h _ { t } = \left( 1 - z _ { t } \right) \circ h _ { t } + z _ { t } \circ h _ { t - 1 }$$

如果$z_t$是单位向量的话，则隐藏层只是复制前一个时刻的隐藏层，梯度不发生变化（衰减）。

这些公式写在一起已经非常直观了：

![屏幕快照 2019-02-10 17.24.43.png](https://i.loli.net/2019/02/10/5c5fedf633978.png)

![屏幕快照 2019-02-10 17.24.58.png](https://i.loli.net/2019/02/10/5c5fedf6b3820.png)

上图中，虚线代表某个gate的调节作用。隐藏层取决于上一个时刻的隐藏层和新的记忆，而update gate负责调节它们的比例，reset gate和输入共同决定新的记忆……

对于短距离的依存来讲，reset gate经常是激活的。

## LSTM（Long-Short-Term-Memories）

### LSTM结构

LSTM与GRU动机相似，只不过单元结构有点不同。GRU的单元结构如下：

![屏幕快照 2019-02-10 17.26.52.png](https://i.loli.net/2019/02/10/5c5fee86c2163.png)

LSTM单元结构如下：

![屏幕快照 2019-02-10 17.27.06.png](https://i.loli.net/2019/02/10/5c5fee86cb45a.png)

上图用公式描述如下：

![屏幕快照 2019-02-10 17.27.12.png](https://i.loli.net/2019/02/10/5c5fee86af964.png)

LSTM的计算可以分为如下步骤

1. New memory generation 与GRU的New memory generation类似。使用当前词语$x_t$和之前的隐状态$h_{t1}$来生成新的记忆$c̃_t$。于是新记忆里面就包含了当前词语$x_{(t)}$的属性。
2. Input Gate 使用当前词语和之前的隐状态决定当前词语是否值得保留来gate新记忆，这个“是否”是通过$i_t$来体现的
3. Forget Gate 与input gate类似，只不过它不是用来衡量输入单词的有用与否，而是衡量过去的记忆对计算当前记忆有用与否。它接受输入单词和上一刻的隐状态产生输出$f_t$。
4. Final memory generation 根据forget gate的建议$f_t$来遗忘过去的记忆$c_{t−1}$。类似地，根据input gate的建议$i_t$来gate信的记忆 $c̃_t$，然后把两者加起来得到最终记忆$c_t$。
5. Output/Exposure Gate 这是GRU中不显式存在的门，用处是将最终记忆与隐状态分离开来。记忆$c_t$中的信息不是全部都需要存放到隐状态中，隐状态是个很重要的使用很频繁的东西，LSTM中每个gate都需要隐状态的参与。Output Gate产生$o_t$，用来gate记忆的tanh激活值。

![屏幕快照 2019-02-10 17.31.25.png](https://i.loli.net/2019/02/10/5c5fef7a4bcb3.png)

### 深度LSTM用于MT

LSTM是序列标注、seq2seq任务的首选模型，可以层叠起来形成更深的模型。在数据量特别大的时候特别有用。

比如与传统MT模型的比较：

![屏幕快照 2019-02-10 17.32.59.png](https://i.loli.net/2019/02/10/5c5ff007a6b63.png)

那时候的NN模型还是仅限于重新排序传统MT模型产生的结果，而最新的研究就是完全甩开了MT模型：

![屏幕快照 2019-02-10 17.33.47.png](https://i.loli.net/2019/02/10/5c5ff007b2e67.png)

前三用的都是NN。

输入原文序列，将最后一个时刻的隐藏层向量PCA降维后可视化：

![屏幕快照 2019-02-10 17.36.01.png](https://i.loli.net/2019/02/10/5c5ff0a2bdb24.png)

发现无论词序如何，意义相同的句子在向量空间中更接近。

### 进一步改进：更多门！

![屏幕快照 2019-02-10 17.36.08.png](https://i.loli.net/2019/02/10/5c5ff0a2c6c88.png)

## RNN的最新改进

### softmax的问题：无法出新词

对分类问题来讲，你无法指望分类模型给你分出一个训练集中不存在的类。即便是训练集中存在的类，如果样本数很少，模型也很难预测出该类。

对于预测下一个单词的语言模型来讲，也是如此。比如某某女士巴拉巴拉，然后自我介绍说我叫某某。如果某某不存在于训练集中，则模型无法预测出某某。

虽然可以用字符级的模型，但代价实在太大。

### 用指针来解决问题

如果把某某替换为“向前数第10个单词”这样的指针，问题就迎刃而解了。

![屏幕快照 2019-02-10 17.38.02.png](https://i.loli.net/2019/02/10/5c5ff11792e11.png)

具体做法是，以前100个时刻的隐藏层作为输入，用一个softmax去计算前100个单词是pointer的概率，与原来的词表上的分布混合。

![屏幕快照 2019-02-10 17.38.09.png](https://i.loli.net/2019/02/10/5c5ff117b79d3.png)

使用了pointer之后，困惑度下降了零点几个百分点：

![屏幕快照 2019-02-10 17.38.16.png](https://i.loli.net/2019/02/10/5c5ff117e8ba3.png)